<a href="https://colab.research.google.com/github/yudintsev-sergey/Finish/blob/main/Mixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Монтируем диск на My Drive

In [ ]:
from google.colab import drive
drive.mount("/ShardDrives/")

Mounted at /ShardDrives/


Импортируем необходимые библиотеки и запускаем отсчет времени, которое алгоритм потратит на выполнение программы

In [ ]:
import librosa                      # пакет для анализа аудиофайлов
from scipy.io.wavfile import write  # пакет анализа аудио со своими особенностями
import os                           # работа с ОС
import numpy as np                  # работа с массивами
import math as ma                   # пакет математических формул
import matplotlib.pyplot as plt     # графики
import time                         # работа с переменными в формате времени

start = time.time()

Определяем рабочую директорию. Там будут все файлы данного проекта.

In [ ]:
os.chdir('/ShardDrives/MyDrive/Colab Notebooks/Out/')

Задаемся константами для Фурье-преобразования (мы будем использовать мел-кепстральный вариант анализа аудиофайла)

In [ ]:
sr = 22050                        # частота сэмплирования (отсчетов / сек)
hop_length = 2048                 # шаг перемещения окна n_fft, в котором происходит мел-кепстральный анализ
n_fft = 512                       # окно FFT (fast Fourie transform)
n_mfcc = 128                      # число мел-интервалов (треугольных фильтров в окне n_fft)
n_common = int(n_mfcc ** 2 + 3)   # размерность вытянутого в линейку массива, полученного на одном шаге hop_length

Из файла .wav, который кажется однообразным, нарезаем в случайные моменты времени одинаковой длины файлы, которые тоже кажутся нам одинаковыми.
Это модель нескольких похожих источников звука, которые находятся на разной удаленности от регистратора.
Для генерации нашей базы берем шум винтов океанского лайнера.

In [ ]:
filename = 'airboat-steady-convert.wav'
y, sr = librosa.load(filename, sr=22050)
drtn = librosa.get_duration(y=y, sr=sr)       # Определяем длительность аудиофайла (сек)
duration = (n_mfcc - 1) * hop_length / sr     # Задаем длительность сэмпла так, чтобы при анализе не было "мертвых зон"
m = 3                                        # Выбираем количество образцов для эксперимента

sounds = []
times = np.random.rand(m) * (drtn - duration)  # Это "m" случайных моментов времени для "биопсии" образцов

for i in range(m):
  y, sr = librosa.load(filename, sr=sr, offset=times[i], duration=duration)
  sounds.append(y)                              # Нарезка коротких звуков одной длительности
  write(str(i) +"_airboat.wav", sr, sounds[i])   # Запись образцов аудио в рабочую директорию

In [ ]:
!pip install pydub                               # Еще один необходимый пакет с дополнительными инструментами анализа
import pydub
from pydub import AudioSegment

Смешиваем файлы по принципу - индекс "0" - беспримесный файл, "1" - смешано два файла, "2" - три файла и т.д.,..., 9 - смешано 10 файлов.

In [ ]:
mixed_sounds = AudioSegment.silent(duration=duration*1000, frame_rate=sr)
mixed_sounds.export("mixed_sounds.wav", format='wav')
for i in range(m):
  sound1 = AudioSegment.from_file('mixed_sounds.wav')
  sound2 = AudioSegment.from_file(str(i) + "_airboat.wav")
  mixed_sounds = sound1.overlay(sound2)
  mixed_sounds.export(str(i) + "_mixed.wav", format='wav')

Константы изменяем

In [ ]:
n_mfcc = 16                       # число мел-интервалов (треугольных фильтров в окне n_fft)
n_common = int(n_mfcc ** 2 + 10)   # размерность вытянутого в линейку массива, полученного на одном шаге hop_length
common_base = np.empty([0, n_common], dtype=float)

Функция нарезки коротких сэмплов для создания базы данных для исследования

In [ ]:
def file_to_base(filename, threeplet=[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
                 sr=22050, hop_length=2048, n_fft=512, n_mfcc=16):
    global common_base              #, sr, hop_length, n_fft, n_mfcc
    duration = 1.0001 * (n_mfcc - 1) * hop_length / sr
    y, sr = librosa.load(filename, sr=sr)
    drtn = librosa.get_duration(y=y, sr=sr)

    off_s = 0.
    while off_s < (drtn - duration):
        y, sr = librosa.load(filename, sr=sr, offset=off_s, duration=duration)
        tmbr = librosa.feature.mfcc(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc)
        tmbr_lin = np.append(tmbr.reshape(1, n_mfcc ** 2), threeplet, axis=1)
        common_base = np.append(common_base, tmbr_lin, axis=0)
        off_s += 0.01 * duration
    return common_base

Берем последовательно все наши 10 образцов и формируем общую базу данных с результатами мел-кепстрального анализа и признаками принадлежности к одному из 10-ти классов.
Время выполнения данного фрагмента кода 105 секунд

In [ ]:
file_to_base('0_mixed.wav', [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
file_to_base('1_mixed.wav', [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])
file_to_base('2_mixed.wav', [[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])
#file_to_base('3_mixed.wav', [[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])
#file_to_base('4_mixed.wav', [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])
#file_to_base('5_mixed.wav', [[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])
#file_to_base('6_mixed.wav', [[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])
#file_to_base('7_mixed.wav', [[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])
#file_to_base('8_mixed.wav', [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])
#file_to_base('9_mixed.wav', [[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

Перемешиваем случайным образом нашу базу данных

In [ ]:
np.random.shuffle(common_base)

Строим 3-D матрицу мел-кепстральных коэффициентов МКК (характеристики тембра), выделяя строки признаков "true" в отдельный файл

In [ ]:
arrays = [common_base[:, :n_mfcc ** 2][i].reshape(n_mfcc,n_mfcc) for i in range(np.shape(common_base)[0])]
tmbr_base = np.stack(arrays, axis=0)
print(tmbr_base.shape)

(2241, 16, 16)


Строим матрицу признаков "true", которые индексированы с матрицей МКК

In [ ]:
true_base = common_base[:, n_mfcc ** 2:]
print(true_base.shape)

(2241, 10)


Сохраняем базы данных в рабочей директории

In [ ]:
np.save('tmbr_base', tmbr_base)
np.save('true_base', true_base)

Справка

In [ ]:
# record end time
end = time.time()
# печатаем длительность работы алгоритма
print("Время выполнения программы = ", (end-start), " сек")

# Справочные данные
print('Размерность "линейного" массива "common_base" = ', np.shape(common_base))
print('Размерность массива "tmbr" = ', np.shape(tmbr_base))
print('Размерность массива признаков "true" = ', np.shape(true_base))
print('Файлы np_array сохранены в родительской папке с расширением *.npy')

Время выполнения программы =  399.2692835330963  сек
Размерность "линейного" массива "common_base" =  (2241, 266)
Размерность массива "tmbr" =  (2241, 16, 16)
Размерность массива признаков "true" =  (2241, 10)
Файлы np_array сохранены в родительской папке с расширением *.npy
